# Code for Sudoku Problem

### Integer programming formulation:

\begin{equation}
\begin{split}
\min~ & 0 \\
s.t. ~& x_{ijy_{ij}} =1, ~ y_{ij}\neq 0\\
& \sum_{i=1}^9x_{ijk} = 1, ~ j,k =1,...,9\\
& \sum_{j=1}^9x_{ijk} = 1, ~ i,k =1,...,9\\
& \sum_{k=1}^9x_{ijk} = 1, ~ i,j =1,...,9\\
& \sum_{i=3p-2}^{3p}\sum_{j=3q-2}^{3q}x_{ijk} = 1, ~ k=1,...,9, p,q =1,2,3\\
& x_{ijk}\in\{0,1\}.
\end{split}
\end{equation}

In [41]:
from gurobipy import *
import numpy as np


#########Parameters Set-up############

y = np.array([[8, 0, 0, 6, 0, 0, 9, 0, 5],
              [0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 2, 0, 3, 1, 0],
              [0, 0, 7, 3, 1, 8, 0, 6, 0],
              [2, 4, 0, 0, 0, 0, 0, 7, 3],
              [0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 2, 7, 9, 0, 1, 0, 0],
              [5, 0, 0, 0, 8, 0, 0, 3, 6], 
              [0, 0, 3, 0, 0, 0, 0, 0, 0]])

print(y)

N = y.shape[0]


[[8 0 0 6 0 0 9 0 5]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 3 1 0]
 [0 0 7 3 1 8 0 6 0]
 [2 4 0 0 0 0 0 7 3]
 [0 0 0 0 0 0 0 0 0]
 [0 0 2 7 9 0 1 0 0]
 [5 0 0 0 8 0 0 3 6]
 [0 0 3 0 0 0 0 0 0]]


In [31]:
#########Model Set-up############

m = Model("Sudoku")

# Creat variables
x = m.addVars(N, N, N, vtype=GRB.BINARY, name = "x")

# Set objective
m.setObjective(0, GRB.MINIMIZE)

# Fill-in constraints:
for i in range(N):
    for j in range(N):
        if y[i,j] > 0:
            m.addConstr( x[i,j, y[i,j]-1] == 1 )

# For every column, each digit appears only once:
m.addConstrs(( quicksum(x[i,j,k] for i in range(N)) == 1  for j in range(N) for k in range(N) ))

# For every row, each digit appears only once:
m.addConstrs(( quicksum(x[i,j,k] for j in range(N)) == 1  for i in range(N) for k in range(N) ))

# For every entry (i,j), only one digit is chosen:
m.addConstrs(( quicksum(x[i,j,k] for k in range(N)) == 1  for i in range(N) for j in range(N) ))

# For 3x3 square, each digit appear only once
m.addConstrs(( quicksum(x[i,j,k] for i in range(3*p, 3*(p+1)) for j in range(3*q, 3*(q+1))) == 1
              for k in range(N) for p in range(3) for q in range(3) ))



# Solving the model
m.optimize()




Optimize a model with 349 rows, 729 columns and 2941 nonzeros
Variable types: 0 continuous, 729 integer (729 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 345 rows and 723 columns
Presolve time: 0.16s
Presolved: 4 rows, 6 columns, 14 nonzeros
Variable types: 0 continuous, 6 integer (6 binary)
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.21 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [40]:
# Print out the solution to the Sudoku problem
for i in range(N):
    for j in range(N):
        for k in range(N):
            if x[i,j,k].x == 1:
                print("%3i" % (k+1), end =" ") 
    print("\n")
    
    

  8   1   4   6   3   7   9   2   5 

  3   2   5   1   4   9   6   8   7 

  7   9   6   8   2   5   3   1   4 

  9   5   7   3   1   8   4   6   2 

  2   4   1   9   5   6   8   7   3 

  6   3   8   2   7   4   5   9   1 

  4   6   2   7   9   3   1   5   8 

  5   7   9   4   8   1   2   3   6 

  1   8   3   5   6   2   7   4   9 



3
4
5
